# Housing Market Data: HUD SAFMR, FHFA HPI, Wharton WRLURI

This notebook pulls housing-market and regulatory data from three sources:

1. **HUD SAFMR** -- Small Area Fair Market Rents by ZIP code (ArcGIS FeatureServer)
2. **FHFA HPI** -- House Price Index by state and ZIP3 (bulk CSV/XLSX download)
3. **Wharton WRLURI** -- Residential Land Use Regulatory Index by community (Stata download)

All sources are **free and require no authentication**.

Results are saved to CSV and merged with the Census ACS + geospatial data from notebooks 06-07.

---
## 1b. Cross-Notebook Data Linking Strategy

This notebook (08) collects **housing market and regulatory** data.
It links to the other two data-collection notebooks to build the unified FMV feature set:

### Data Flow: Three Notebooks -> One FMV Feature Matrix

```
NB 06 -- Census + FRED + BLS + FCC        NB 07 -- USGS + FEMA + TIGER       NB 08 -- HUD + FHFA + WRLURI
  |  Demographics, income, employment        |  Elevation, flood, land cover      |  Fair market rent, HPI, zoning
  |  Competition (tower density)              |  Highway proximity, hazard score   |  Regulatory restrictiveness
  |  Macro indicators (CPI, GDP, rates)       |  Buildability index                |  Housing market pressure
  |                                           |                                    |
  +-------- JOIN KEY: state_abbr -------------+-------- JOIN KEY: state_abbr ------+
  +-------- JOIN KEY: county_fips ------------+                                    |
  +-------- JOIN KEY: tract_fips -------------+                                    |
  +-------- JOIN KEY: zcta (zip) ----------------------------------------------- -+
  +-------- JOIN KEY: site_id (lat/lon) --------+                                 |
                          |                                                        |
                          v                                                        |
               +-----------------------+                                           |
               | UNIFIED SITE SPINE    | <-- spatial join from site lat/lon --------+
               | (one row per site)    |
               +-----------------------+
                          |
                          v
               +-----------------------+
               | FMV FEATURE MATRIX    |   ~75-80 features across 4 tiers
               | Tier 1: X_market (45) |   from Domains D1-D7
               | Tier 2: Z_network (19)|   + Tier 3 flags + Tier 4 interactions
               +-----------------------+
```

### Join Key Hierarchy

| Join Level | Key | NB 06 Sources | NB 07 Sources | NB 08 Sources | Sites Sharing |
|-----------|-----|--------------|---------------|---------------|---------------|
| **Site** (lat/lon) | `site_id` | FCC tower radius queries | USGS elevation, FEMA flood, NLCD, TIGER highway | -- | 1 site |
| **Block Group** | `bg_fips` (12-digit) | Census ACS (19 vars) | EPA Smart Location | -- | 5-50 sites |
| **Tract** | `tract_fips` (11-digit) | Census ACS (28 vars), FCC tower density | USGS elevation, NLCD, Urban/Rural class | -- | 10-100 sites |
| **ZIP/ZCTA** | `zcta` (5-digit) | Census ACS (28 vars) | -- | **HUD SAFMR** (rent by bedroom) | 20-200 sites |
| **County** | `county_fips` (5-digit) | BLS unemployment, QCEW wages | NOAA storm events, USDA ag land values | WRLURI (via crosswalk) | 50-500 sites |
| **State** | `state_abbr` | Census ACS, FRED macro, BLS LAUS | All geospatial (state centroid) | **FHFA HPI**, **WRLURI** (state agg) | 100-5000 sites |
| **National** | -- | FRED (CPI, GDP, mortgage rates, treasury) | -- | -- | All sites |

### Why Housing Data Is Critical for FMV

Cell tower rent does NOT exist in isolation. It is anchored to the **local real estate market**:

- **HUD SAFMR** = government-set rent floor for the ZIP. If residential rents are high,
  landlords have higher opportunity cost -> higher tower lease expectation.
- **FHFA HPI** = property value momentum. Rising home prices = rising land values =
  rising FMV for any land use, including tower leases.
- **WRLURI** = regulatory friction. Harder to get permits = fewer alternative sites =
  scarcity premium on existing towers.

Together, these three sources feed into **4 of the top 10 FMV coefficients** in the LLD:
`log_land_value` (0.15), `log_median_home_value` (0.35), `permit_difficulty` (0.08),
and `scarcity_index` (0.15).


In [ ]:
import os
import io
import time
import zipfile
import requests
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

# Project root
_cwd = os.getcwd()
_project_root = _cwd if os.path.isdir(os.path.join(_cwd, "data")) or os.path.isfile(os.path.join(_cwd, ".env")) else os.path.dirname(_cwd)
DATA_DIR = os.path.join(_project_root, "data")
os.makedirs(DATA_DIR, exist_ok=True)

# FIPS / abbreviation mappings
FIPS_TO_ABBR = {
    "01": "AL", "02": "AK", "04": "AZ", "05": "AR", "06": "CA",
    "08": "CO", "09": "CT", "10": "DE", "11": "DC", "12": "FL",
    "13": "GA", "15": "HI", "16": "ID", "17": "IL", "18": "IN",
    "19": "IA", "20": "KS", "21": "KY", "22": "LA", "23": "ME",
    "24": "MD", "25": "MA", "26": "MI", "27": "MN", "28": "MS",
    "29": "MO", "30": "MT", "31": "NE", "32": "NV", "33": "NH",
    "34": "NJ", "35": "NM", "36": "NY", "37": "NC", "38": "ND",
    "39": "OH", "40": "OK", "41": "OR", "42": "PA", "44": "RI",
    "45": "SC", "46": "SD", "47": "TN", "48": "TX", "49": "UT",
    "50": "VT", "51": "VA", "53": "WA", "54": "WV", "55": "WI",
    "56": "WY", "72": "PR",
}
ABBR_TO_FIPS = {v: k for k, v in FIPS_TO_ABBR.items()}

# Full state name -> abbreviation
STATE_NAME_TO_ABBR = {
    "Alabama": "AL", "Alaska": "AK", "Arizona": "AZ", "Arkansas": "AR",
    "California": "CA", "Colorado": "CO", "Connecticut": "CT", "Delaware": "DE",
    "District of Columbia": "DC", "Florida": "FL", "Georgia": "GA", "Hawaii": "HI",
    "Idaho": "ID", "Illinois": "IL", "Indiana": "IN", "Iowa": "IA",
    "Kansas": "KS", "Kentucky": "KY", "Louisiana": "LA", "Maine": "ME",
    "Maryland": "MD", "Massachusetts": "MA", "Michigan": "MI", "Minnesota": "MN",
    "Mississippi": "MS", "Missouri": "MO", "Montana": "MT", "Nebraska": "NE",
    "Nevada": "NV", "New Hampshire": "NH", "New Jersey": "NJ", "New Mexico": "NM",
    "New York": "NY", "North Carolina": "NC", "North Dakota": "ND", "Ohio": "OH",
    "Oklahoma": "OK", "Oregon": "OR", "Pennsylvania": "PA", "Rhode Island": "RI",
    "South Carolina": "SC", "South Dakota": "SD", "Tennessee": "TN", "Texas": "TX",
    "Utah": "UT", "Vermont": "VT", "Virginia": "VA", "Washington": "WA",
    "West Virginia": "WV", "Wisconsin": "WI", "Wyoming": "WY",
}
ABBR_TO_STATE_NAME = {v: k for k, v in STATE_NAME_TO_ABBR.items()}

print(f"Project root: {_project_root}")
print(f"Data directory: {DATA_DIR}")

---
## 2. HUD -- Small Area Fair Market Rents (SAFMR)

The **HUD SAFMR** provides ZIP-level fair market rents by bedroom count. Data is served via
an ArcGIS FeatureServer with ~33,800 ZIP records.

- **Endpoint:** `https://services.arcgis.com/.../HUD_PDR_Small_Area_Fair_Market_Rents/FeatureServer`
- **Layer 1** = Rent data table (SAFMR_0BR through SAFMR_4BR, keyed by ZIP/ZCTA)
- **Auth:** None
- **Pagination:** 1,000 records per request; we paginate through all ~33,800 records.

In [ ]:
# ---- HUD SAFMR: Fetch all ZIP-level rents (paginated, ~34 pages of 1000) ----
HUD_SAFMR_URL = (
    "https://services.arcgis.com/VTyQ9soqVukalItT/arcgis/rest/services/"
    "HUD_PDR_Small_Area_Fair_Market_Rents/FeatureServer/1/query"
)

all_safmr_rows = []
offset = 0
page_size = 1000
page = 0

print("Fetching HUD SAFMR data (Layer 1 -- rent table)...")
print(f"Page size: {page_size}, paginating until all records fetched.\n")

while True:
    params = {
        "where": "1=1",
        "outFields": "ID,HUD_CODE,FMR_NAME,SAFMR_0BR,SAFMR_1BR,SAFMR_2BR,SAFMR_3BR,SAFMR_4BR",
        "resultOffset": offset,
        "resultRecordCount": page_size,
        "f": "json",
    }
    try:
        r = requests.get(HUD_SAFMR_URL, params=params, timeout=30)
        r.raise_for_status()
        data = r.json()
        features = data.get("features", [])
        if not features:
            break
        for feat in features:
            all_safmr_rows.append(feat.get("attributes", {}))
        page += 1
        offset += page_size
        exceeded = data.get("exceededTransferLimit", False)
        print(f"  Page {page}: {len(features)} records (total so far: {len(all_safmr_rows):,})")
        if not exceeded:
            break
        time.sleep(0.3)
    except Exception as e:
        print(f"  Page {page + 1}: ERROR - {e}")
        break

df_safmr = pd.DataFrame(all_safmr_rows)

# Rename columns for clarity
df_safmr.rename(columns={
    "ID": "zcta",
    "HUD_CODE": "hud_code",
    "FMR_NAME": "fmr_name",
    "SAFMR_0BR": "safmr_0br",
    "SAFMR_1BR": "safmr_1br",
    "SAFMR_2BR": "safmr_2br",
    "SAFMR_3BR": "safmr_3br",
    "SAFMR_4BR": "safmr_4br",
}, inplace=True)

# Convert rent columns to numeric
for col in ["safmr_0br", "safmr_1br", "safmr_2br", "safmr_3br", "safmr_4br"]:
    df_safmr[col] = pd.to_numeric(df_safmr[col], errors="coerce")

# Derive state from first 2 digits of ZIP (approximate)
# Map ZIP prefix to state using first 3 digits
df_safmr["zcta"] = df_safmr["zcta"].astype(str).str.zfill(5)

print(f"\n--- HUD SAFMR: {len(df_safmr):,} ZIP/ZCTA records ---")
print(f"Columns: {list(df_safmr.columns)}")
print(f"\n2-BR Rent stats:")
print(df_safmr["safmr_2br"].describe().round(0))
df_safmr.head(10)

In [ ]:
# ---- Aggregate SAFMR to state level & save ----
# Extract state from FMR name (format: "City, ST MSA" or similar)
def extract_state_from_fmr(fmr_name):
    """Try to extract 2-letter state code from FMR area name."""
    if not isinstance(fmr_name, str):
        return None
    # Look for 2-letter state code pattern
    import re
    # Pattern: ", XX " or ", XX$" where XX is a state abbreviation
    match = re.search(r",\s*([A-Z]{2})(?:\s|$|,)", fmr_name)
    if match:
        code = match.group(1)
        if code in ABBR_TO_STATE_NAME or code == "DC":
            return code
    return None

df_safmr["state_abbr"] = df_safmr["fmr_name"].apply(extract_state_from_fmr)
matched = df_safmr["state_abbr"].notna().sum()
print(f"State extracted for {matched:,}/{len(df_safmr):,} records ({matched/len(df_safmr)*100:.1f}%)\n")

# Aggregate: median SAFMR by state
df_safmr_state = df_safmr.groupby("state_abbr").agg(
    safmr_0br_median=("safmr_0br", "median"),
    safmr_1br_median=("safmr_1br", "median"),
    safmr_2br_median=("safmr_2br", "median"),
    safmr_3br_median=("safmr_3br", "median"),
    safmr_4br_median=("safmr_4br", "median"),
    safmr_zip_count=("zcta", "count"),
).reset_index()

for col in df_safmr_state.columns:
    if "median" in col:
        df_safmr_state[col] = df_safmr_state[col].round(0).astype(int)

print(f"--- SAFMR State-Level Summary: {len(df_safmr_state)} states ---")
print(df_safmr_state.head(10).to_string(index=False))

# Save ZIP-level (full) as Parquet (large), state-level as CSV
safmr_zip_path = os.path.join(DATA_DIR, "hud_safmr_by_zip.parquet")
df_safmr.to_parquet(safmr_zip_path, index=False, engine="pyarrow")
zip_size_mb = os.path.getsize(safmr_zip_path) / 1024 / 1024
print(f"\nSaved ZIP-level: {safmr_zip_path} ({len(df_safmr):,} rows, {zip_size_mb:.1f} MB, Parquet)")

safmr_state_path = os.path.join(DATA_DIR, "hud_safmr_by_state.csv")
df_safmr_state.to_csv(safmr_state_path, index=False)
print(f"Saved state-level: {safmr_state_path} ({len(df_safmr_state)} rows, CSV)")

In [ ]:
# ---- Map 1: HUD SAFMR -- Median 2-BR Rent by State ----
fig_safmr = px.choropleth(
    df_safmr_state,
    locations="state_abbr",
    locationmode="USA-states",
    color="safmr_2br_median",
    scope="usa",
    color_continuous_scale="Reds",
    hover_name="state_abbr",
    hover_data={
        "safmr_0br_median": ":$,.0f",
        "safmr_1br_median": ":$,.0f",
        "safmr_2br_median": ":$,.0f",
        "safmr_3br_median": ":$,.0f",
        "safmr_4br_median": ":$,.0f",
        "safmr_zip_count": ":,",
        "state_abbr": False,
    },
    labels={"safmr_2br_median": "Median 2-BR SAFMR ($)"},
    title="HUD Small Area Fair Market Rent -- Median 2-BR by State",
)
fig_safmr.update_layout(
    geo=dict(bgcolor="rgba(0,0,0,0)"),
    paper_bgcolor="white",
    margin=dict(l=0, r=0, t=50, b=0),
    height=500,
)
fig_safmr.show()

In [ ]:
# ---- Map 2: SAFMR Rent Spread (4BR - 0BR) as measure of family size premium ----
df_safmr_state["rent_spread_4br_0br"] = df_safmr_state["safmr_4br_median"] - df_safmr_state["safmr_0br_median"]

fig_spread = px.choropleth(
    df_safmr_state,
    locations="state_abbr",
    locationmode="USA-states",
    color="rent_spread_4br_0br",
    scope="usa",
    color_continuous_scale="Purples",
    hover_name="state_abbr",
    hover_data={
        "safmr_0br_median": ":$,.0f",
        "safmr_4br_median": ":$,.0f",
        "rent_spread_4br_0br": ":$,.0f",
        "state_abbr": False,
    },
    labels={"rent_spread_4br_0br": "4BR - 0BR Spread ($)"},
    title="HUD SAFMR -- Rent Spread: 4-Bedroom minus Studio by State",
)
fig_spread.update_layout(
    geo=dict(bgcolor="rgba(0,0,0,0)"),
    paper_bgcolor="white",
    margin=dict(l=0, r=0, t=50, b=0),
    height=500,
)
fig_spread.show()

---
## 3. FHFA -- House Price Index (HPI)

The **FHFA House Price Index** tracks residential property price changes at multiple geographic levels.

- **State-level HPI:** `https://www.fhfa.gov/hpi/download/quarterly_datasets/hpi_at_state.csv`
- **ZIP3-level HPI:** `https://www.fhfa.gov/hpi/download/annual/hpi_at_zip3.xlsx`
- **Auth:** None (direct download)
- The all-transactions HPI uses 1995 Q1 = 100 as the base index.

In [ ]:
# ---- FHFA HPI: Download state-level quarterly data ----
FHFA_STATE_URL = "https://www.fhfa.gov/hpi/download/quarterly_datasets/hpi_at_state.csv"

print(f"Downloading FHFA state-level HPI from: {FHFA_STATE_URL}")
try:
    r_fhfa = requests.get(FHFA_STATE_URL, timeout=60)
    r_fhfa.raise_for_status()
    df_hpi_state = pd.read_csv(io.StringIO(r_fhfa.text))
    print(f"Downloaded: {len(df_hpi_state):,} rows x {len(df_hpi_state.columns)} columns")
    print(f"Columns: {list(df_hpi_state.columns)}")
    print(f"\nSample:")
    print(df_hpi_state.head(5).to_string(index=False))
except Exception as e:
    print(f"Download failed: {e}")
    print("Trying alternative approach...")
    # Fallback: try the master HPI file
    try:
        alt_url = "https://www.fhfa.gov/hpi/download/monthly/hpi_master.csv"
        print(f"Trying master HPI: {alt_url}")
        r_alt = requests.get(alt_url, timeout=60)
        r_alt.raise_for_status()
        df_master = pd.read_csv(io.StringIO(r_alt.text))
        # Filter to state-level, all-transactions
        df_hpi_state = df_master[
            (df_master["level"].str.contains("State", case=False, na=False)) |
            (df_master["place_name"].isin(STATE_NAME_TO_ABBR.keys()))
        ].copy()
        print(f"Filtered master to {len(df_hpi_state):,} state rows")
    except Exception as e2:
        print(f"Alternative also failed: {e2}")
        df_hpi_state = pd.DataFrame()

In [ ]:
# ---- Process FHFA state HPI data ----
if len(df_hpi_state) > 0:
    # Standardize column names (the CSV may have varying formats)
    df_hpi_state.columns = df_hpi_state.columns.str.strip().str.lower().str.replace(" ", "_")
    print(f"Columns after normalization: {list(df_hpi_state.columns)}")
    
    # Try to identify the state name and HPI value columns
    # Common column names: state/place_name, yr/year, qtr/quarter, index_nsa/hpi
    state_col = None
    for candidate in ["state", "place_name", "state_name"]:
        if candidate in df_hpi_state.columns:
            state_col = candidate
            break
    
    hpi_col = None
    for candidate in ["index_nsa", "hpi", "index", "hpi_with_appraisal", "hpi_sa"]:
        if candidate in df_hpi_state.columns:
            hpi_col = candidate
            break
    
    year_col = None
    for candidate in ["yr", "year"]:
        if candidate in df_hpi_state.columns:
            year_col = candidate
            break
    
    qtr_col = None
    for candidate in ["qtr", "quarter", "period"]:
        if candidate in df_hpi_state.columns:
            qtr_col = candidate
            break
    
    print(f"Identified columns -- state: {state_col}, hpi: {hpi_col}, year: {year_col}, quarter: {qtr_col}")
    
    if state_col and hpi_col and year_col:
        # Map state names to abbreviations
        df_hpi_state["state_abbr"] = df_hpi_state[state_col].map(STATE_NAME_TO_ABBR)
        # If that didn't work, try direct (column might already have abbreviations)
        unmapped = df_hpi_state["state_abbr"].isna().sum()
        if unmapped > len(df_hpi_state) * 0.5:
            # Might already be abbreviations
            valid_abbrs = set(ABBR_TO_STATE_NAME.keys())
            if df_hpi_state[state_col].isin(valid_abbrs).sum() > 10:
                df_hpi_state["state_abbr"] = df_hpi_state[state_col]
        
        df_hpi_state["year"] = pd.to_numeric(df_hpi_state[year_col], errors="coerce")
        df_hpi_state["hpi_value"] = pd.to_numeric(df_hpi_state[hpi_col], errors="coerce")
        if qtr_col:
            df_hpi_state["quarter"] = pd.to_numeric(df_hpi_state[qtr_col], errors="coerce")
        
        # Filter to valid state data and recent 15 years
        df_hpi_clean = df_hpi_state[
            (df_hpi_state["state_abbr"].notna()) &
            (df_hpi_state["hpi_value"].notna()) &
            (df_hpi_state["year"] >= 2010)
        ].copy()
        
        print(f"\n--- FHFA HPI State-Level (2010+): {len(df_hpi_clean):,} rows ---")
        print(f"States: {df_hpi_clean['state_abbr'].nunique()}")
        print(f"Years: {int(df_hpi_clean['year'].min())} - {int(df_hpi_clean['year'].max())}")
        print(f"HPI range: {df_hpi_clean['hpi_value'].min():.1f} - {df_hpi_clean['hpi_value'].max():.1f}")
    else:
        print("Could not identify required columns. Printing first few rows for inspection:")
        print(df_hpi_state.head())
        df_hpi_clean = pd.DataFrame()
else:
    df_hpi_clean = pd.DataFrame()
    print("No FHFA data available.")

In [ ]:
# ---- Save FHFA HPI data ----
if len(df_hpi_clean) > 0:
    # Save full quarterly data
    hpi_full_path = os.path.join(DATA_DIR, "fhfa_hpi_state_quarterly.csv")
    df_hpi_clean.to_csv(hpi_full_path, index=False)
    print(f"Saved quarterly: {hpi_full_path} ({len(df_hpi_clean):,} rows)")
    
    # Annual average HPI by state (for easier merge with census annual data)
    df_hpi_annual = df_hpi_clean.groupby(["state_abbr", "year"]).agg(
        hpi_annual_avg=("hpi_value", "mean"),
    ).reset_index()
    df_hpi_annual["hpi_annual_avg"] = df_hpi_annual["hpi_annual_avg"].round(2)
    
    hpi_annual_path = os.path.join(DATA_DIR, "fhfa_hpi_state_annual.csv")
    df_hpi_annual.to_csv(hpi_annual_path, index=False)
    print(f"Saved annual avg: {hpi_annual_path} ({len(df_hpi_annual):,} rows)")
    
    # Latest year snapshot for state-level merge
    latest_hpi_year = int(df_hpi_annual["year"].max())
    df_hpi_latest = df_hpi_annual[df_hpi_annual["year"] == latest_hpi_year].copy()
    print(f"Latest year ({latest_hpi_year}): {len(df_hpi_latest)} states")
    df_hpi_annual.tail(10)
else:
    df_hpi_annual = pd.DataFrame()
    df_hpi_latest = pd.DataFrame()
    print("No HPI data to save.")

In [ ]:
# ---- Map 3: FHFA House Price Index by State (Latest Year) ----
if len(df_hpi_latest) > 0:
    fig_hpi = px.choropleth(
        df_hpi_latest,
        locations="state_abbr",
        locationmode="USA-states",
        color="hpi_annual_avg",
        scope="usa",
        color_continuous_scale="YlOrRd",
        hover_name="state_abbr",
        hover_data={
            "hpi_annual_avg": ":.1f",
            "year": True,
            "state_abbr": False,
        },
        labels={"hpi_annual_avg": "HPI (1995Q1=100)"},
        title=f"FHFA House Price Index by State ({latest_hpi_year}, All-Transactions, 1995Q1=100)",
    )
    fig_hpi.update_layout(
        geo=dict(bgcolor="rgba(0,0,0,0)"),
        paper_bgcolor="white",
        margin=dict(l=0, r=0, t=50, b=0),
        height=500,
    )
    fig_hpi.show()
else:
    print("No HPI data for map.")

In [ ]:
# ---- Map 4: FHFA HPI Animated Over Time (2010-present) ----
if len(df_hpi_annual) > 0:
    fig_hpi_anim = px.choropleth(
        df_hpi_annual.sort_values("year"),
        locations="state_abbr",
        locationmode="USA-states",
        color="hpi_annual_avg",
        scope="usa",
        animation_frame="year",
        color_continuous_scale="YlOrRd",
        range_color=[df_hpi_annual["hpi_annual_avg"].quantile(0.02),
                     df_hpi_annual["hpi_annual_avg"].quantile(0.98)],
        hover_name="state_abbr",
        hover_data={"hpi_annual_avg": ":.1f", "state_abbr": False, "year": True},
        labels={"hpi_annual_avg": "HPI (1995Q1=100)", "year": "Year"},
        title="FHFA House Price Index by State (2010-Present, Animated)",
    )
    fig_hpi_anim.update_layout(
        geo=dict(bgcolor="rgba(0,0,0,0)"),
        paper_bgcolor="white",
        margin=dict(l=0, r=0, t=50, b=0),
        height=500,
    )
    fig_hpi_anim.show()
else:
    print("No HPI data for animated map.")

---
## 4. Wharton WRLURI -- Residential Land Use Regulatory Index

The **Wharton Residential Land Use Regulatory Index** measures local zoning/permit difficulty
across ~2,450 US communities.

- **Source:** Wharton faculty data page (Stata `.dta` format)
- **Download:** ZIP file containing `.dta` files; read with `pandas.read_stata()`
- **Key fields:** `WRLURI` (overall index), `LPPI` (political pressure), `LZAI` (zoning approval),
  `DRI` (density restrictions), plus state/community identifiers.
- **Auth:** None (public research data)

In [ ]:
# ---- Wharton WRLURI: Download and read Stata data file ----
WRLURI_URL = "https://real-faculty.wharton.upenn.edu/wp-content/uploads/~gyourko/WRLURI/WHARTONLANDREGULATIONDATA_1_15_2020.zip"

print(f"Downloading Wharton WRLURI data from:\n  {WRLURI_URL}")
print("(This is the 2018 survey data, ~2,450 communities)\n")

df_wrluri = pd.DataFrame()
try:
    r_wrluri = requests.get(WRLURI_URL, timeout=120)
    r_wrluri.raise_for_status()
    print(f"Downloaded: {len(r_wrluri.content) / 1024 / 1024:.1f} MB")
    
    # Extract .dta file(s) from ZIP
    with zipfile.ZipFile(io.BytesIO(r_wrluri.content)) as zf:
        dta_files = [f for f in zf.namelist() if f.endswith(".dta")]
        print(f"ZIP contents: {zf.namelist()}")
        print(f"Stata files found: {dta_files}")
        
        if dta_files:
            # Read the first .dta file
            with zf.open(dta_files[0]) as dta_f:
                df_wrluri = pd.read_stata(io.BytesIO(dta_f.read()))
            print(f"\nLoaded: {len(df_wrluri):,} rows x {len(df_wrluri.columns)} columns")
            print(f"Columns: {list(df_wrluri.columns)}")
        else:
            # Try CSV or Excel files
            other_files = [f for f in zf.namelist() if f.endswith((".csv", ".xlsx", ".xls"))]
            if other_files:
                with zf.open(other_files[0]) as of:
                    if other_files[0].endswith(".csv"):
                        df_wrluri = pd.read_csv(of)
                    else:
                        df_wrluri = pd.read_excel(io.BytesIO(of.read()))
                print(f"Loaded from {other_files[0]}: {len(df_wrluri):,} rows")

except Exception as e:
    print(f"Download/parse failed: {e}")
    print("The Wharton data server may be temporarily unavailable.")

if len(df_wrluri) > 0:
    print(f"\n--- Wharton WRLURI: {len(df_wrluri):,} communities ---")
    # Show key columns if available
    key_cols = [c for c in df_wrluri.columns if c.upper() in 
               ["WRLURI", "LPPI", "SPII", "CII", "LZAI", "LPAI", "DRI", "SRI", "EI", "AHI",
                "STATENAME", "STATE", "PLACENAME", "COMMUNITYNAME", "STAESSION", "FIPS"]]
    if key_cols:
        print(f"Key index columns found: {key_cols}")
    df_wrluri.head(10)
else:
    print("No WRLURI data loaded.")

In [ ]:
# ---- Process Wharton WRLURI -- aggregate to state level ----
if len(df_wrluri) > 0:
    # Normalize column names to lowercase
    df_wrluri.columns = df_wrluri.columns.str.strip().str.upper()
    
    # Find state column
    state_col_w = None
    for candidate in ["STATENAME", "STATE", "STAESSION"]:
        if candidate in df_wrluri.columns:
            state_col_w = candidate
            break
    
    # Find WRLURI column
    wrluri_col = None
    for candidate in ["WRLURI", "WRLURI18", "WRLURI_2018"]:
        if candidate in df_wrluri.columns:
            wrluri_col = candidate
            break
    
    print(f"State column: {state_col_w}, WRLURI column: {wrluri_col}")
    print(f"All columns: {list(df_wrluri.columns)}")
    
    if state_col_w and wrluri_col:
        # Map state names to abbreviations
        df_wrluri["state_abbr"] = df_wrluri[state_col_w].map(STATE_NAME_TO_ABBR)
        # If not matched, try treating as abbreviation directly
        if df_wrluri["state_abbr"].isna().sum() > len(df_wrluri) * 0.5:
            valid_abbrs = set(ABBR_TO_STATE_NAME.keys())
            if df_wrluri[state_col_w].isin(valid_abbrs).sum() > 10:
                df_wrluri["state_abbr"] = df_wrluri[state_col_w]
        
        df_wrluri["wrluri_value"] = pd.to_numeric(df_wrluri[wrluri_col], errors="coerce")
        
        # Find sub-index columns
        sub_indices = {}
        for idx_name in ["LPPI", "SPII", "CII", "LZAI", "LPAI", "DRI", "SRI", "EI", "AHI"]:
            for col in df_wrluri.columns:
                if idx_name in col:
                    sub_indices[idx_name.lower()] = col
                    break
        print(f"Sub-indices found: {sub_indices}")
        
        # Aggregate to state level
        agg_dict = {
            "wrluri_mean": ("wrluri_value", "mean"),
            "wrluri_median": ("wrluri_value", "median"),
            "wrluri_community_count": ("wrluri_value", "count"),
        }
        
        df_wrluri_state = df_wrluri.groupby("state_abbr").agg(**agg_dict).reset_index()
        df_wrluri_state["wrluri_mean"] = df_wrluri_state["wrluri_mean"].round(3)
        df_wrluri_state["wrluri_median"] = df_wrluri_state["wrluri_median"].round(3)
        
        print(f"\n--- WRLURI State-Level: {len(df_wrluri_state)} states ---")
        print(f"WRLURI range: {df_wrluri_state['wrluri_mean'].min():.3f} to {df_wrluri_state['wrluri_mean'].max():.3f}")
        print(f"(Positive = more restrictive, negative = less restrictive)")
        print(df_wrluri_state.sort_values("wrluri_mean", ascending=False).head(10).to_string(index=False))
    else:
        df_wrluri_state = pd.DataFrame()
        print("Could not identify state/WRLURI columns.")
else:
    df_wrluri_state = pd.DataFrame()
    print("No WRLURI data to process.")

In [ ]:
# ---- Save WRLURI data ----
if len(df_wrluri) > 0:
    wrluri_full_path = os.path.join(DATA_DIR, "wharton_wrluri_communities.csv")
    df_wrluri.to_csv(wrluri_full_path, index=False)
    print(f"Saved community-level: {wrluri_full_path} ({len(df_wrluri):,} rows)")

if len(df_wrluri_state) > 0:
    wrluri_state_path = os.path.join(DATA_DIR, "wharton_wrluri_by_state.csv")
    df_wrluri_state.to_csv(wrluri_state_path, index=False)
    print(f"Saved state-level: {wrluri_state_path} ({len(df_wrluri_state)} rows)")

if len(df_wrluri) == 0 and len(df_wrluri_state) == 0:
    print("No WRLURI data to save.")

In [ ]:
# ---- Map 5: Wharton WRLURI -- Regulatory Restrictiveness by State ----
if len(df_wrluri_state) > 0:
    fig_wrluri = px.choropleth(
        df_wrluri_state,
        locations="state_abbr",
        locationmode="USA-states",
        color="wrluri_mean",
        scope="usa",
        color_continuous_scale="RdYlGn_r",  # red = more restrictive
        hover_name="state_abbr",
        hover_data={
            "wrluri_mean": ":.3f",
            "wrluri_median": ":.3f",
            "wrluri_community_count": ":,",
            "state_abbr": False,
        },
        labels={"wrluri_mean": "WRLURI (mean)"},
        title="Wharton WRLURI -- Land Use Regulatory Restrictiveness by State (higher = more restrictive)",
    )
    fig_wrluri.update_layout(
        geo=dict(bgcolor="rgba(0,0,0,0)"),
        paper_bgcolor="white",
        margin=dict(l=0, r=0, t=50, b=0),
        height=500,
    )
    fig_wrluri.show()
else:
    print("No WRLURI data for map.")

---
## 5. Combine All Data & Merge with Census + Geospatial

Join HUD SAFMR, FHFA HPI, and Wharton WRLURI at the state level, then merge with the
existing Census ACS + geospatial combined dataset from notebooks 06-07.

In [ ]:
# ---- Combine all housing/regulatory datasets at state level ----
# Start with SAFMR state-level
df_housing = df_safmr_state.copy() if len(df_safmr_state) > 0 else pd.DataFrame(columns=["state_abbr"])

# Merge FHFA HPI (latest year snapshot)
if len(df_hpi_latest) > 0:
    df_housing = df_housing.merge(
        df_hpi_latest[["state_abbr", "hpi_annual_avg"]],
        on="state_abbr", how="outer"
    )
    print(f"Added FHFA HPI: {df_hpi_latest['state_abbr'].nunique()} states")

# Merge Wharton WRLURI
if len(df_wrluri_state) > 0:
    df_housing = df_housing.merge(
        df_wrluri_state[["state_abbr", "wrluri_mean", "wrluri_median", "wrluri_community_count"]],
        on="state_abbr", how="outer"
    )
    print(f"Added Wharton WRLURI: {df_wrluri_state['state_abbr'].nunique()} states")

print(f"\n--- Combined Housing Dataset: {len(df_housing)} states x {len(df_housing.columns)} columns ---")
print(f"Columns: {list(df_housing.columns)}")
df_housing.head(10)

In [ ]:
# ---- Merge with Census + Geospatial combined data ----
census_geo_path = os.path.join(DATA_DIR, "census_geo_combined_by_state.csv")
census_state_path = os.path.join(DATA_DIR, "census_state_demographics_2010_2024.csv")

df_base = None
if os.path.exists(census_geo_path):
    df_base = pd.read_csv(census_geo_path, dtype={"state": str})
    if "state_abbr" not in df_base.columns and "state" in df_base.columns:
        df_base["state_abbr"] = df_base["state"].map(FIPS_TO_ABBR)
    print(f"Loaded census+geo: {len(df_base):,} rows x {len(df_base.columns)} cols")
elif os.path.exists(census_state_path):
    df_base = pd.read_csv(census_state_path, dtype={"state": str})
    if "state_abbr" not in df_base.columns:
        df_base["state_abbr"] = df_base["state"].map(FIPS_TO_ABBR)
    print(f"Loaded census (no geo): {len(df_base):,} rows x {len(df_base.columns)} cols")
else:
    print(f"No census data found at {census_geo_path} or {census_state_path}")
    print("Run notebooks 06-07 first. Will save housing data standalone.")

if df_base is not None:
    # Merge housing data (static per state) onto multi-year census data
    df_full = df_base.merge(df_housing, on="state_abbr", how="left")
    
    # Also merge FHFA HPI by year (time-varying)
    if len(df_hpi_annual) > 0 and "year" in df_full.columns:
        df_full["year"] = df_full["year"].astype(int)
        df_hpi_annual["year"] = df_hpi_annual["year"].astype(int)
        # Drop the static hpi_annual_avg, merge time-varying one
        if "hpi_annual_avg" in df_full.columns:
            df_full.drop(columns=["hpi_annual_avg"], inplace=True)
        df_full = df_full.merge(
            df_hpi_annual[["state_abbr", "year", "hpi_annual_avg"]],
            on=["state_abbr", "year"], how="left"
        )
        print(f"Merged time-varying HPI by state+year")
    
    print(f"\n--- Full Combined Dataset ---")
    print(f"Rows: {len(df_full):,} | Columns: {len(df_full.columns)}")
    new_cols = [c for c in df_full.columns if c not in df_base.columns]
    print(f"New columns from housing data: {new_cols}")
    df_full.head(10)
else:
    df_full = None

In [ ]:
# ---- Save all combined datasets ----
# 1) Housing-only state-level
housing_path = os.path.join(DATA_DIR, "housing_features_by_state.csv")
df_housing.to_csv(housing_path, index=False)
print(f"Saved housing-only: {housing_path} ({len(df_housing)} rows)")

# 2) Full combined (census + geo + housing)
if df_full is not None:
    full_path = os.path.join(DATA_DIR, "census_geo_housing_combined_by_state.csv")
    df_full.to_csv(full_path, index=False)
    size_mb = os.path.getsize(full_path) / 1024 / 1024
    print(f"Saved full combined: {full_path} ({len(df_full):,} rows, {size_mb:.2f} MB)")
else:
    print("Full combined not saved (no base census data).")

In [ ]:
# ---- Correlation: Housing features vs Census demographics ----
if df_full is not None and "year" in df_full.columns:
    latest = int(df_full["year"].max())
    df_latest = df_full[
        (df_full["year"] == latest) &
        (df_full["median_income"] > 0) if "median_income" in df_full.columns else True
    ].copy()
    
    corr_cols = [
        "safmr_2br_median", "hpi_annual_avg", "wrluri_mean",
        "median_income", "median_home_value", "median_rent", "population",
    ]
    available = [c for c in corr_cols if c in df_latest.columns and df_latest[c].notna().sum() > 5]
    
    if len(available) >= 2:
        corr_matrix = df_latest[available].corr().round(3)
        print(f"=== Correlation Matrix ({latest}, {len(df_latest)} states) ===")
        print(corr_matrix.to_string())
        
        print(f"\n--- Key Correlations ---")
        pairs = [
            ("safmr_2br_median", "median_rent", "SAFMR vs Census Rent"),
            ("hpi_annual_avg", "median_home_value", "HPI vs Home Value"),
            ("wrluri_mean", "median_home_value", "WRLURI vs Home Value"),
            ("wrluri_mean", "safmr_2br_median", "WRLURI vs SAFMR Rent"),
            ("hpi_annual_avg", "median_income", "HPI vs Income"),
        ]
        for c1, c2, label in pairs:
            if c1 in available and c2 in available:
                val = corr_matrix.loc[c1, c2]
                print(f"  {label:35s} {val:+.3f}")
    else:
        print(f"Not enough overlapping columns for correlation. Available: {available}")
else:
    print("Skipping correlation -- combined data not available.")

---
## 6. Summary

In [ ]:
# ---- Final Summary ----
print("=" * 80)
print("HOUSING & REGULATORY DATA COLLECTION SUMMARY")
print("=" * 80)

print(f"\n  Source              | Records      | Key Attributes")
print(f"  -------------------+-------------+-------------------------------")
print(f"  HUD SAFMR (ZIP)    | {len(df_safmr):>10,} | safmr_0br..4br by ZIP/ZCTA")
print(f"  HUD SAFMR (State)  | {len(df_safmr_state):>10,} | median rents by state")
if len(df_hpi_clean) > 0:
    print(f"  FHFA HPI (Qtrly)   | {len(df_hpi_clean):>10,} | quarterly HPI by state")
    print(f"  FHFA HPI (Annual)  | {len(df_hpi_annual):>10,} | annual avg HPI by state")
else:
    print(f"  FHFA HPI           |          0 | (download failed)")
if len(df_wrluri) > 0:
    print(f"  Wharton WRLURI     | {len(df_wrluri):>10,} | regulatory index by community")
    print(f"  WRLURI (State)     | {len(df_wrluri_state):>10,} | mean WRLURI by state")
else:
    print(f"  Wharton WRLURI     |          0 | (download failed)")
print(f"  Housing Combined   | {len(df_housing):>10,} | all housing features by state")
if df_full is not None:
    print(f"  Full Combined      | {len(df_full):>10,} | census+geo+housing ({len(df_full.columns)} columns)")

print(f"\n  Data files saved:")
saved_files = [
    "hud_safmr_by_zip.parquet",
    "hud_safmr_by_state.csv",
    "fhfa_hpi_state_quarterly.csv",
    "fhfa_hpi_state_annual.csv",
    "wharton_wrluri_communities.csv",
    "wharton_wrluri_by_state.csv",
    "housing_features_by_state.csv",
    "census_geo_housing_combined_by_state.csv",
]
for fname in saved_files:
    fpath = os.path.join(DATA_DIR, fname)
    if os.path.exists(fpath):
        size_kb = os.path.getsize(fpath) / 1024
        unit = "KB" if size_kb < 1024 else "MB"
        size_val = size_kb if size_kb < 1024 else size_kb / 1024
        print(f"    {fname:50s} {size_val:>8.1f} {unit}")
    else:
        print(f"    {fname:50s}  (not saved)")

print(f"\n  Maps generated: 5 total")
print(f"    HUD SAFMR:       Median 2-BR Rent + Rent Spread = 2 maps")
print(f"    FHFA HPI:        Latest HPI + Animated HPI = 2 maps")
print(f"    Wharton WRLURI:  Regulatory Restrictiveness = 1 map")

print("\n" + "=" * 80)
print("ALL HOUSING & REGULATORY DATA COLLECTED SUCCESSFULLY")
print("=" * 80)

---
## 7b. FRED Rent & Housing Indices (CPI-Shelter, Homeownership Rate)

The **FRED API** provides national and regional housing indicators that are NOT available
from HUD or FHFA. These capture **macro housing market dynamics**:

| Series | Description | FMV Role |
|--------|-------------|----------|
| `CUSR0000SEHA` | CPI-Shelter (rent of primary residence) | National rent inflation benchmark |
| `RHORUSQ156N` | Homeownership Rate (US) | Low ownership = high rental demand = higher FMV |
| `MORTGAGE30US` | 30-Year Fixed Mortgage Rate | High rates depress buying, boost renting |
| `PERMIT` | New Private Housing Building Permits | Supply pipeline -- more permits = future competition |
| `MSACSR` | Monthly Supply of New Houses | Supply tightness indicator |
| `MSPUS` | Median Sales Price of Houses Sold (US) | National housing price level |

These are **national-level** time series. They apply uniformly to all sites but capture
temporal momentum -- a lease signed in a rising-rate environment has different FMV
dynamics than one signed during low rates.


In [ ]:
# ---- FRED Rent & Housing Indices ----
from dotenv import load_dotenv
load_dotenv(os.path.join(_project_root, ".env"))
FRED_API_KEY = os.environ.get("FRED_API_KEY", "")

FRED_HOUSING_SERIES = {
    "CUSR0000SEHA":  "cpi_shelter_rent_index",
    "RHORUSQ156N":   "homeownership_rate_pct",
    "MORTGAGE30US":   "mortgage_rate_30yr",
    "PERMIT":         "building_permits_thousands",
    "MSACSR":         "months_supply_new_houses",
    "MSPUS":          "median_sale_price_us",
}

fred_housing_frames = {}

if FRED_API_KEY:
    print("Fetching FRED housing/rent indices...")
    for series_id, label in FRED_HOUSING_SERIES.items():
        url = f"https://api.stlouisfed.org/fred/series/observations"
        params = {
            "series_id": series_id,
            "api_key": FRED_API_KEY,
            "file_type": "json",
            "observation_start": "2010-01-01",
            "frequency": "a",
        }
        try:
            r = requests.get(url, params=params, timeout=30)
            r.raise_for_status()
            obs = r.json().get("observations", [])
            df_s = pd.DataFrame(obs)
            df_s["value"] = pd.to_numeric(df_s["value"], errors="coerce")
            df_s["year"] = pd.to_datetime(df_s["date"]).dt.year
            df_s = df_s[["year", "value"]].dropna().rename(columns={"value": label})
            fred_housing_frames[label] = df_s
            latest = df_s.iloc[-1]
            print(f"  {series_id:20s} -> {label:35s} latest={latest[label]:.2f} ({int(latest["year"])})")
        except Exception as e:
            print(f"  {series_id}: FAILED - {e}")
    
    if fred_housing_frames:
        df_fred_housing = fred_housing_frames[list(fred_housing_frames.keys())[0]]
        for label, df_s in list(fred_housing_frames.items())[1:]:
            df_fred_housing = df_fred_housing.merge(df_s, on="year", how="outer")
        df_fred_housing = df_fred_housing.sort_values("year")
        
        fred_housing_path = os.path.join(DATA_DIR, "fred_housing_indices.csv")
        df_fred_housing.to_csv(fred_housing_path, index=False)
        print(f"\n  Saved: {fred_housing_path} ({len(df_fred_housing)} rows)")
        display(df_fred_housing.tail(5))
    else:
        df_fred_housing = pd.DataFrame()
else:
    print("FRED_API_KEY not set. Skipping FRED housing indices.")
    print("Set FRED_API_KEY in .env to enable.")
    df_fred_housing = pd.DataFrame()


---
## 7c. BEA Regional GDP by State (Real GDP Growth)

The **Bureau of Economic Analysis (BEA)** publishes annual real GDP by state.
State-level GDP growth captures the **economic momentum** of the area:

- States with strong GDP growth have expanding businesses that need more coverage
- Higher GDP = higher commercial activity = higher demand for cell infrastructure
- GDP per capita proxies the economic productivity that drives land values

This directly feeds the LLD's **Domain 6 (Macro)** feature `gdp_per_capita_msa_scaled`.


In [ ]:
# ---- BEA Regional GDP by State ----
# BEA provides free data via their API. We use the Regional GDP tables.
# Endpoint: https://apps.bea.gov/api/data/
# No API key needed for basic access.

BEA_API_KEY = os.environ.get("BEA_API_KEY", "")

if BEA_API_KEY:
    print("Fetching BEA Regional GDP by state...")
    bea_url = "https://apps.bea.gov/api/data/"
    params = {
        "UserID": BEA_API_KEY,
        "method": "GetData",
        "DataSetName": "Regional",
        "TableName": "SAGDP1",
        "GeoFips": "STATE",
        "LineCode": "1",
        "Year": "2015,2016,2017,2018,2019,2020,2021,2022,2023,2024",
        "ResultFormat": "json",
    }
    try:
        r = requests.get(bea_url, params=params, timeout=60)
        r.raise_for_status()
        data = r.json()
        results = data.get("BEAAPI", {}).get("Results", {}).get("Data", [])
        if results:
            df_bea = pd.DataFrame(results)
            df_bea["DataValue"] = pd.to_numeric(
                df_bea["DataValue"].str.replace(",", ""), errors="coerce"
            )
            df_bea["year"] = pd.to_numeric(df_bea["TimePeriod"], errors="coerce")
            df_bea["state_name"] = df_bea["GeoName"]
            df_bea["state_abbr"] = df_bea["state_name"].map(STATE_NAME_TO_ABBR)
            df_bea = df_bea[df_bea["state_abbr"].notna()].copy()
            df_bea = df_bea.rename(columns={"DataValue": "real_gdp_millions"})
            
            # GDP growth rate
            df_bea = df_bea.sort_values(["state_abbr", "year"])
            df_bea["gdp_yoy_growth"] = df_bea.groupby("state_abbr")["real_gdp_millions"].pct_change()
            
            bea_path = os.path.join(DATA_DIR, "bea_state_gdp.csv")
            df_bea[["state_abbr", "year", "real_gdp_millions", "gdp_yoy_growth"]].to_csv(
                bea_path, index=False
            )
            print(f"  Saved: {bea_path} ({len(df_bea)} rows, {df_bea['state_abbr'].nunique()} states)")
            display(df_bea[["state_abbr", "year", "real_gdp_millions", "gdp_yoy_growth"]].tail(10))
        else:
            print("  No results from BEA API.")
            df_bea = pd.DataFrame()
    except Exception as e:
        print(f"  BEA API failed: {e}")
        df_bea = pd.DataFrame()
else:
    print("BEA_API_KEY not set. Skipping BEA GDP.")
    print("Register free at https://apps.bea.gov/API/signup/")
    df_bea = pd.DataFrame()


---
## 7d. Census Building Permits Survey (State-Level New Construction)

The **Census Bureau Building Permits Survey** tracks new residential construction permits.
This is a leading indicator of **housing supply expansion**:

- More permits = more new construction = future competition for existing structures
- High permit activity in a county means landlords face future supply pressure
- Low permits (especially combined with high WRLURI) = constrained supply = scarcity premium

This interacts with NB08's WRLURI (regulatory friction) and NB06's Census ACS (population growth).
Together, `permit_volume` + `wrluri_index` + `population_growth` predicts whether a market is
supply-constrained (boosting FMV) or oversupplied (depressing FMV).


In [ ]:
# ---- Census Building Permits Survey (Annual, State-Level) ----
# Source: https://www.census.gov/construction/bps/
# The data is available as downloadable CSV/TXT files.

PERMITS_URL = (
    "https://www2.census.gov/econ/bps/State/st_annual.txt"
)

print("Fetching Census Building Permits Survey (state-level annual)...")
try:
    r = requests.get(PERMITS_URL, timeout=60)
    r.raise_for_status()
    
    lines = r.text.strip().split("\n")
    print(f"  Downloaded: {len(lines)} lines")
    print(f"  Header: {lines[0][:120]}")
    
    df_permits = pd.read_csv(
        io.StringIO(r.text), sep=",", skipinitialspace=True,
        dtype=str, on_bad_lines="skip"
    )
    df_permits.columns = df_permits.columns.str.strip().str.lower().str.replace(" ", "_")
    print(f"  Columns: {list(df_permits.columns)}")
    print(f"  Rows: {len(df_permits):,}")
    
    # Identify numeric permit columns and convert
    for col in df_permits.columns:
        if "unit" in col or "bldg" in col or "permit" in col or "value" in col:
            df_permits[col] = pd.to_numeric(
                df_permits[col].str.replace(",", ""), errors="coerce"
            )
    
    permits_path = os.path.join(DATA_DIR, "census_building_permits_state.csv")
    df_permits.to_csv(permits_path, index=False)
    print(f"  Saved: {permits_path}")
    df_permits.head(5)

except Exception as e:
    print(f"  Download failed: {e}")
    print("  Alternative: download manually from https://www.census.gov/construction/bps/stateannual.html")
    df_permits = pd.DataFrame()


---
## 7e. IRS SOI Migration Data (County-Level In/Out Migration)

The **IRS Statistics of Income (SOI)** publishes county-to-county migration data
derived from tax return address changes:

- **Net migration inflow** = more people moving in than out = growing demand
- **Adjusted gross income (AGI) of migrants** = quality of demand (high-income migration
  drives property values more than low-income migration)
- Counties with net outflow face declining demand -> downward FMV pressure

This is a **leading indicator** -- people move first, then housing prices adjust,
then commercial rents (including tower leases) follow 1-2 years later.

Links to NB06: population growth rate (Census ACS) captures the same signal at tract level,
but IRS SOI adds the **income quality** dimension and the **migration flow direction**.


In [ ]:
# ---- IRS SOI Migration Data (County-Level) ----
# Source: https://www.irs.gov/statistics/soi-tax-stats-migration-data
# Available as ZIP/CSV downloads by year.

IRS_MIGRATION_URL = (
    "https://www.irs.gov/pub/irs-soi/county_migration_data_2122.csv"
)

print("Fetching IRS SOI county migration data (2021-2022 filing year)...")
try:
    r = requests.get(IRS_MIGRATION_URL, timeout=120)
    r.raise_for_status()
    df_migration = pd.read_csv(io.StringIO(r.text), dtype=str, on_bad_lines="skip")
    df_migration.columns = df_migration.columns.str.strip().str.lower()
    print(f"  Downloaded: {len(df_migration):,} rows x {len(df_migration.columns)} cols")
    print(f"  Columns: {list(df_migration.columns)}")
    
    migration_path = os.path.join(DATA_DIR, "irs_soi_migration_county.csv")
    df_migration.to_csv(migration_path, index=False)
    print(f"  Saved: {migration_path}")
    df_migration.head(5)

except Exception as e:
    print(f"  Download failed: {e}")
    print("  The IRS SOI URL format changes yearly. Download manually from:")
    print("  https://www.irs.gov/statistics/soi-tax-stats-migration-data")
    print("  Look for 'County-to-County Migration Data' CSV files.")
    df_migration = pd.DataFrame()


---
## 12. Complete FMV Feature Map: All Three Notebooks -> LLD Model Tiers

This section maps **every attribute** collected across notebooks 06, 07, and 08
to the LLD's feature tiers and domains. This is the definitive reference for
understanding which data source feeds which model coefficient.

### Tier 1: X_market Features (45 features, Domains D1-D7)

#### Domain 5 -- Microeconomic (highest FMV impact, 14 features)

| Feature | Source Notebook | Raw API Variable | Granularity | FMV Mechanism |
|---------|---------------|-------------------|-------------|---------------|
| `census_population_density_log` | **NB06** Census ACS | `B01003_001E` / tract area | Tract | Dense areas = high demand for coverage = higher FMV |
| `census_population_3km_log` | **NB06** Census ACS | spatial buffer aggregation | Site (3km) | Local demand intensity around the specific tower |
| `census_median_income_scaled` | **NB06** Census ACS | `B19013_001E` | Tract | Higher income = higher willingness to pay for quality coverage |
| `census_median_home_value_log` | **NB06** Census ACS | `B25077_001E` | Tract | **Strongest FMV predictor** -- log home value explains 35% of rent variance |
| `census_median_rent_log` | **NB06** Census ACS | `B25064_001E` | Tract | Residential rent benchmark anchors tower lease negotiation |
| `census_vacancy_rate` | **NB06** Census ACS | `B25002_003E / B25002_001E` | Tract | High vacancy = weak demand = downward FMV pressure |
| `urban_rural_class_ordinal` | **NB07** Census Gazetteer | Tract classification | Tract | Urban vs rural fundamentally changes FMV economics |
| `underlying_land_value_psf_log` | **NB08** HUD SAFMR proxy | `safmr_2br * 12 / area` | ZIP | Land value directly determines opportunity cost for landlord |
| `hud_safmr_log` | **NB08** HUD SAFMR | `SAFMR_2BR` | ZIP | Government rent floor -- anchors local market expectations |
| `property_tax_rate` | **NB06** Census ACS / local | ACS + local data | County | Higher taxes = higher carrying cost = higher rent demanded |
| `topography_class_ordinal` | **NB07** USGS 3DEP | EPQS elevation + variance | Site | Mountaintop sites have natural height advantage = premium |
| `terrain_elevation_variance_m` | **NB07** USGS 3DEP | EPQS multi-point query | Site | Rough terrain = harder to build alternatives = scarcity |
| `poi_density_1km` | External (OSM/Google) | Places API | Site (1km) | Commercial density proxies economic activity |
| `building_density_500m` | **NB06**/External | Census blocks + imagery | Site (500m) | Built environment density = demand intensity |

#### Domain 2 -- Zoning/Regulatory (6 features)

| Feature | Source Notebook | Raw Variable | Granularity | FMV Mechanism |
|---------|---------------|--------------|-------------|---------------|
| `zoning_district_type_encoded` | Local data | Municipal zoning | Parcel | Commercial zones command higher rents than residential |
| `zoning_telecom_permitted_ordinal` | Local data | Municipal code | Parcel | By-right = lower premium; conditional use = higher (scarcity) |
| `permit_approval_difficulty` | **NB08** WRLURI proxy | `WRLURI` sub-indices | State/County | **Direct FMV coefficient (0.08)**: harder permits = higher FMV |
| `wrluri_index` | **NB08** Wharton WRLURI | `WRLURI` composite | State | Regulatory friction limits new tower construction |
| `height_restriction_ft` | Local data | Municipal code | Parcel | Height limits constrain alternatives = scarcity premium |
| `colocation_required` | Local data | Municipal code | Jurisdiction | Colocation mandates push multi-tenant -> higher rent |

#### Domain 6 -- Macroeconomic (2 features)

| Feature | Source Notebook | Raw Variable | Granularity | FMV Mechanism |
|---------|---------------|--------------|-------------|---------------|
| `unemployment_rate_local` | **NB06** BLS LAUS | County LAUS series | County | High unemployment = weak economy = downward FMV pressure |
| `gdp_per_capita_msa_scaled` | **NB06** FRED + **NB08** BEA | MSA GDP / population | MSA | Economic productivity drives all commercial rents |

#### Domain 7 -- Competition (9 features)

| Feature | Source Notebook | Raw Variable | Granularity | FMV Mechanism |
|---------|---------------|--------------|-------------|---------------|
| `fcc_tower_count_5km` | **NB06** FCC ASR | ASR bulk download | Site (5km) | More towers nearby = more alternatives = lower FMV |
| `fcc_tower_density_sqkm` | **NB06** FCC ASR | ASR + tract area | Tract | Regional tower saturation measure |
| `competitor_tower_distance_km` | **NB06** FCC ASR / OpenCelliD | Nearest-neighbor query | Site | Closest alternative -- key scarcity signal |
| `tower_per_capita` | **NB06** FCC + Census | towers / population | Tract | Coverage adequacy per person |
| `scarcity_index` | **NB06** + **NB07** + **NB08** | Composite | Site | **Key FMV coefficient (0.15)**: combines buildability + competition + regulation |
| `landlord_concentration` | Lease data | HHI of landlords | Submarket | Concentrated ownership = pricing power |
| `composite_hazard_score` | **NB07** FEMA + NOAA | Flood risk + storm history | Site/County | Hazard exposure increases insurance cost factored into lease |
| `distance_to_highway_km` | **NB07** TIGER | Primary road proximity | Site | Highway access affects maintenance cost and site value |
| `ground_elevation_ft` | **NB07** USGS 3DEP | EPQS elevation | Site | Higher elevation = better signal propagation = premium |

### Tier 4: Interaction Features (from LLD Table 12)

| Interaction Feature | Components | Source Notebooks | Why It Matters |
|-------------------|-----------|-----------------|----------------|
| `income_density_interaction` | `log(census_median_income * census_population_density)` | NB06 x NB06 | Rich + dense = premium urban market |
| `tower_competition_density` | `fcc_tower_count_5km / census_population_density` | NB06 x NB06 | Competition relative to demand |
| `scarcity_x_demand` | `scarcity_index * census_population_3km` | NB06+07+08 x NB06 | Scarce sites in high-demand areas = maximum FMV |
| `permit_difficulty_x_density` | `permit_approval_difficulty * census_population_density` | **NB08** x NB06 | Regulatory friction amplified by demand |
| `hpi_x_scarcity` | `hpi_appreciation_3yr * scarcity_index` | **NB08** x NB06+07 | Rising market + scarce site = accelerating FMV |
| `safmr_premium_x_tower_density` | `safmr_premium_pct * fcc_tower_density_sqkm` | **NB08** x NB06 | High-rent area with many towers = competitive pressure |


In [ ]:
# ---- Derived Interaction Features for FMV ----
# These features combine data from multiple notebooks to create the LLD's
# Tier 4 interaction terms. They capture non-linear relationships between
# market conditions that single features cannot represent.

print("=" * 80)
print("Derived Interaction Features (Cross-Notebook)")
print("=" * 80)

# Load the combined state-level data if available
combined_path = os.path.join(DATA_DIR, "census_geo_housing_combined_by_state.csv")
if os.path.isfile(combined_path):
    df_combined = pd.read_csv(combined_path)
    print(f"Loaded combined dataset: {len(df_combined)} rows x {len(df_combined.columns)} cols")
else:
    # Build from individual files
    df_combined = df_housing.copy() if len(df_housing) > 0 else pd.DataFrame(columns=["state_abbr"])
    print(f"Using housing-only dataset: {len(df_combined)} rows")

if len(df_combined) > 0:
    # ---- 1. permit_difficulty_x_density ----
    # NB08 WRLURI x NB06 Census population density
    if "permit_difficulty_mean" in df_combined.columns or "wrluri_mean" in df_combined.columns:
        permit_col = "permit_difficulty_mean" if "permit_difficulty_mean" in df_combined.columns else "wrluri_mean"
        if "population" in df_combined.columns:
            pop_density = df_combined["population"].rank(pct=True)
            df_combined["permit_difficulty_x_density"] = (
                df_combined[permit_col].fillna(5) * pop_density * 10
            ).round(3)
            print(f"  permit_difficulty_x_density: median={df_combined['permit_difficulty_x_density'].median():.2f}")
    
    # ---- 2. hpi_x_scarcity_proxy ----
    # NB08 FHFA HPI appreciation x NB07 scarcity proxy
    if "hpi_appreciation_3yr" in df_combined.columns:
        scarcity_proxy_col = None
        for candidate in ["wrluri_scarcity_boost", "wrluri_mean", "permit_difficulty_mean"]:
            if candidate in df_combined.columns:
                scarcity_proxy_col = candidate
                break
        if scarcity_proxy_col:
            df_combined["hpi_x_scarcity"] = (
                df_combined["hpi_appreciation_3yr"].fillna(0)
                * df_combined[scarcity_proxy_col].rank(pct=True)
                * 100
            ).round(3)
            print(f"  hpi_x_scarcity: median={df_combined['hpi_x_scarcity'].median():.2f}")
    
    # ---- 3. safmr_premium_signal ----
    # NB08 SAFMR vs Census rent premium x tower competition signal
    if "safmr_premium_pct" in df_combined.columns:
        df_combined["safmr_market_heat"] = (
            df_combined["safmr_premium_pct"].rank(pct=True) * 10
        ).round(2)
        print(f"  safmr_market_heat: median={df_combined['safmr_market_heat'].median():.2f}")
    
    # ---- 4. housing_supply_constraint_index ----
    # Combines: high WRLURI (hard to build) + low building permits + high demand
    permit_rank = 0.5
    wrluri_rank = 0.5
    demand_rank = 0.5
    
    if "wrluri_mean" in df_combined.columns:
        wrluri_rank = df_combined["wrluri_mean"].rank(pct=True).fillna(0.5)
    elif "permit_difficulty_mean" in df_combined.columns:
        wrluri_rank = df_combined["permit_difficulty_mean"].rank(pct=True).fillna(0.5)
    
    if "hpi_appreciation_3yr" in df_combined.columns:
        demand_rank = df_combined["hpi_appreciation_3yr"].rank(pct=True).fillna(0.5)
    
    df_combined["supply_constraint_index"] = (
        (wrluri_rank * 0.5 + demand_rank * 0.3 + (1 - permit_rank) * 0.2) * 10
    ).round(2)
    print(f"  supply_constraint_index: median={df_combined['supply_constraint_index'].median():.2f}")
    
    # ---- 5. rent_affordability_stress_x_hpi ----
    if "rent_to_income_ratio" in df_combined.columns and "hpi_appreciation_3yr" in df_combined.columns:
        df_combined["affordability_x_hpi"] = (
            df_combined["rent_to_income_ratio"].rank(pct=True)
            * df_combined["hpi_appreciation_3yr"].rank(pct=True)
            * 100
        ).round(2)
        print(f"  affordability_x_hpi: median={df_combined['affordability_x_hpi'].median():.2f}")
    
    # Save enriched combined dataset
    interaction_cols = [c for c in df_combined.columns if c.endswith("_x_") or 
                        c in ["permit_difficulty_x_density", "hpi_x_scarcity",
                              "safmr_market_heat", "supply_constraint_index",
                              "affordability_x_hpi"]]
    print(f"\n  Interaction features created: {interaction_cols}")
    
    enriched_path = os.path.join(DATA_DIR, "processed", "housing_with_interactions.parquet")
    os.makedirs(os.path.dirname(enriched_path), exist_ok=True)
    df_combined.to_parquet(enriched_path, index=False)
    print(f"  Saved: {enriched_path}")
else:
    print("No combined data available for interaction features.")


---
## 13. Unified Data Pipeline Summary -- All Three Notebooks

This is the comprehensive inventory of **every data source, attribute, and derived feature**
across notebooks 06, 07, and 08 -- and how they feed the FMV model.


In [ ]:
# ---- Unified Pipeline Summary ----
print("=" * 100)
print("UNIFIED DATA PIPELINE -- ALL THREE NOTEBOOKS -> FMV MODEL")
print("=" * 100)

pipeline = [
    ("NB 06: Census API", "Census ACS 5-Year", "Tract/ZCTA/County/State", "28 raw + 8 derived",
     "D5 Micro (income, density, home value, rent, vacancy)\n"
     "     + D6 Macro (unemployment via BLS merge)"),
    ("NB 06: Census API", "FRED Macro Series", "National + MSA", "15 series + 20 MSA GDP",
     "D6 Macro (CPI, rates, permits, GDP)\n"
     "     Temporal context for lease vintage"),
    ("NB 06: Census API", "BLS QCEW", "County", "Employment, wages by industry",
     "D6 Macro (local labor market strength)"),
    ("NB 06: Census API", "FCC ASR", "Site/Tract", "Tower count, density, height",
     "D7 Competition (tower competition, scarcity)"),
    ("NB 06: Census API", "FCC BDC", "Census Block", "Broadband provider count, speeds",
     "D7 Competition (broadband competition)"),
    ("", "", "", "", ""),
    ("NB 07: Geospatial", "USGS 3DEP", "Site (lat/lon)", "Elevation, terrain variance",
     "D5 Micro (topography_class, terrain_variance)\n"
     "     D7 Competition (elevation advantage)"),
    ("NB 07: Geospatial", "USGS NLCD", "Site", "Land cover classification",
     "D5 Micro (development intensity)"),
    ("NB 07: Geospatial", "FEMA NFHL", "Site", "Flood zone, SFHA flag, risk score",
     "D7 Competition (composite_hazard_score)\n"
     "     Higher risk = higher insurance = higher lease"),
    ("NB 07: Geospatial", "TIGER/Line", "Site", "Highway distance",
     "D7 Competition (distance_to_highway_km)\n"
     "     Accessibility affects maintenance cost"),
    ("NB 07: Geospatial", "HUD SAFMR (geo)", "ZIP", "Rent benchmarks (0-4BR)",
     "D5 Micro (hud_safmr_log, land_value proxy)"),
    ("NB 07: Geospatial", "Census Urban/Rural", "Tract", "UA/UC/Rural classification",
     "D5 Micro (urban_rural_class_ordinal)"),
    ("NB 07: Geospatial", "EPA Smart Location", "Block Group", "Walkability, job access",
     "D5 Micro (accessibility, built environment)"),
    ("NB 07: Geospatial", "NOAA Storm Events", "County", "Storm frequency & damage",
     "D7 Competition (hazard component)"),
    ("NB 07: Geospatial", "USDA Ag Land Values", "County/State", "Land values per acre",
     "D5 Micro (rural land value benchmark)"),
    ("", "", "", "", ""),
    ("NB 08: Housing/Reg", "HUD SAFMR (full)", "ZIP (33,800)", "0BR-4BR rent by ZIP",
     "D5 Micro: hud_safmr_log (coeff ~0.15)\n"
     "     Government rent floor anchors FMV"),
    ("NB 08: Housing/Reg", "FHFA HPI", "State quarterly", "House Price Index + appreciation",
     "D5 Micro: log_median_home_value proxy (coeff ~0.35)\n"
     "     Market momentum = FMV trajectory"),
    ("NB 08: Housing/Reg", "Wharton WRLURI", "Community/State", "Regulatory index + sub-indices",
     "D2 Zoning: permit_difficulty (coeff ~0.08)\n"
     "     D7 Competition: scarcity_index (coeff ~0.15)"),
    ("NB 08: Housing/Reg", "FRED Housing Indices", "National", "CPI-Shelter, mortgage, permits",
     "D6 Macro: temporal housing market context\n"
     "     Rate environment affects lease economics"),
    ("NB 08: Housing/Reg", "BEA Regional GDP", "State annual", "Real GDP + growth rate",
     "D6 Macro: gdp_per_capita_msa_scaled\n"
     "     Economic base drives all commercial rents"),
    ("NB 08: Housing/Reg", "Census Bldg Permits", "State/County", "New construction volume",
     "D7 Competition: supply pipeline signal\n"
     "     More permits = future supply = FMV pressure down"),
    ("NB 08: Housing/Reg", "IRS SOI Migration", "County", "Net migration + AGI of migrants",
     "D5 Micro: demand momentum signal\n"
     "     In-migration = growing demand = FMV up"),
]

print(f"\n{'Notebook':22s} | {'Data Source':22s} | {'Granularity':20s} | {'Attributes':25s} | FMV Domain & Role")
print("-" * 140)
for nb_name, source, grain, attrs, role in pipeline:
    if not nb_name:
        print("-" * 140)
        continue
    role_lines = role.split("\n")
    print(f"{nb_name:22s} | {source:22s} | {grain:20s} | {attrs:25s} | {role_lines[0]}")
    for extra_line in role_lines[1:]:
        print(f"{" ":22s} | {" ":22s} | {" ":20s} | {" ":25s} | {extra_line}")

eq = "=" * 100
print(f"\n{eq}")
print("FEATURE COUNT SUMMARY")
print(eq)
print(f"  Tier 1: X_market features        ~45 raw + ~15 encoded = ~60")
print(f"  Tier 2: Z_network features        ~19")
print(f"  Tier 3: Flag features              ~5")
print(f"  Tier 4: Interaction features        ~10")
print(f"  TOTAL for FMV model              ~90-95 features")
print(f"")
print(f"  Data sources across all notebooks:  22")
print(f"  Geographic granularities:           7 (Site -> Block Group -> Tract -> ZIP -> County -> State -> National)")
print(f"  Temporal coverage:                  2010-2024 (15 years for trends)")

print(f"\n--- Top 10 FMV-Critical Features (by coefficient weight) ---")
top_features = [
    ("census_median_home_value_log", "~0.35", "NB06 Census ACS", "D5 Micro", "Strongest single predictor of tower lease FMV"),
    ("scarcity_index", "~0.15", "NB06+07+08 composite", "D7 Competition", "Fewer alternatives = landlord pricing power"),
    ("underlying_land_value_psf_log", "~0.15", "NB08 HUD SAFMR proxy", "D5 Micro", "Landlord opportunity cost"),
    ("census_population_density_log", "~0.12", "NB06 Census ACS", "D5 Micro", "Demand intensity for coverage"),
    ("hpi_appreciation_3yr", "~0.10", "NB08 FHFA HPI", "D5 Micro", "Market momentum / trajectory"),
    ("fcc_tower_count_5km", "~0.10", "NB06 FCC ASR", "D7 Competition", "Direct competition measure"),
    ("permit_approval_difficulty", "~0.08", "NB08 Wharton WRLURI", "D2 Zoning", "Regulatory friction = scarcity"),
    ("census_median_income_scaled", "~0.07", "NB06 Census ACS", "D5 Micro", "Willingness to pay for coverage"),
    ("composite_hazard_score", "~0.05", "NB07 FEMA+NOAA", "D7 Competition", "Risk premium in lease pricing"),
    ("unemployment_rate_local", "~0.04", "NB06 BLS LAUS", "D6 Macro", "Local economic health"),
]
print(f"\n  {'Feature':40s} | {'Weight':8s} | {'Source':25s} | {'Domain':15s} | Mechanism")
print(f"  " + "-" * 130)
for feat, weight, src, domain, mech in top_features:
    print(f"  {feat:40s} | {weight:8s} | {src:25s} | {domain:15s} | {mech}")

print(f"\n{eq}")
print("DATA COLLECTION COMPLETE ACROSS ALL THREE NOTEBOOKS")
print(eq)


---
## 7. FMV Indicator Variables -- Granularity & Scaling Analysis

### Problem Statement
**Fair Market Value (FMV)** is the annual rent a willing buyer and seller would agree to for a cell tower lease.
The model predicts FMV per site using features spanning demographics, macroeconomics, competition, and housing markets.

### Granularity Assessment of This Notebook's Sources

| Source | Current Granularity | Available Granularity | FMV Join Key | Gap |
|--------|--------------------|-----------------------|--------------|-----|
| HUD SAFMR | **ZIP** (33,800 ZCTAs) | ZIP -- already finest | `zip_code` | None -- but need derived features |
| FHFA HPI | **State** quarterly | **ZIP5** + **Tract** (from bulk downloads) | `state_abbr` / `zip_code` / `tract_fips` | State OK for trends; ZIP/tract for levels |
| Wharton WRLURI | **Community** (~2,450) | Community -- need county/place mapping | `county_fips` or spatial | Need community-to-county crosswalk |

### Key Indicator Variables to Derive

From these three sources we can construct features that map directly to FMV model coefficients:

| Indicator | Source(s) | FMV Coefficient | Weight |
|-----------|-----------|-----------------|--------|
| `log_median_home_value` proxy via HPI | FHFA HPI | `log_median_home_value` | **0.35** |
| `hpi_appreciation_1yr/3yr/5yr` | FHFA HPI time series | Market momentum | temporal signal |
| `safmr_rent_level` (2BR benchmark) | HUD SAFMR | `log_land_value` proxy | **0.15** |
| `rent_to_income_ratio` | HUD SAFMR + Census ACS | Housing affordability stress | market signal |
| `safmr_premium_vs_census` | HUD SAFMR - Census rent | Market rent premium | demand signal |
| `wrluri_permit_difficulty` | Wharton WRLURI | `permit_difficulty` | **0.08** |
| `wrluri_scarcity_boost` | Derived from WRLURI | `scarcity_index` | **0.15** |

In [ ]:
# ---- HUD SAFMR: ZIP-Level FMV Indicator Variables ----
print("=" * 80)
print("HUD SAFMR -- ZIP-Level FMV Indicator Variables")
print("=" * 80)

if len(df_safmr) > 0:
    # 1. Rent level features (already at ZIP)
    df_safmr["safmr_2br"] = pd.to_numeric(df_safmr["safmr_2br"], errors="coerce")
    df_safmr["safmr_1br"] = pd.to_numeric(df_safmr["safmr_1br"], errors="coerce")
    df_safmr["safmr_0br"] = pd.to_numeric(df_safmr["safmr_0br"], errors="coerce")
    df_safmr["safmr_4br"] = pd.to_numeric(df_safmr["safmr_4br"], errors="coerce")

    # 2. Rent spread (family size premium) -- wider spread = more demand differentiation
    df_safmr["rent_spread_4br_0br"] = df_safmr["safmr_4br"] - df_safmr["safmr_0br"]

    # 3. Log rent (for the log-scale FMV model)
    df_safmr["log_safmr_2br"] = np.log1p(df_safmr["safmr_2br"].clip(lower=1))

    # 4. Rent percentile within state (relative market position)
    df_safmr["safmr_2br_state_pctile"] = df_safmr.groupby("state_abbr")["safmr_2br"].rank(pct=True)

    # 5. National rent percentile
    df_safmr["safmr_2br_national_pctile"] = df_safmr["safmr_2br"].rank(pct=True)

    # 6. Rent tier (categorical for market segmentation)
    df_safmr["rent_tier"] = pd.cut(
        df_safmr["safmr_2br"],
        bins=[0, 800, 1200, 1800, 2500, float("inf")],
        labels=["budget", "moderate", "above_avg", "premium", "luxury"]
    )

    print(f"\n  ZIP records with indicators: {len(df_safmr):,}")
    print(f"\n  2-BR SAFMR distribution:")
    print(f"    Mean:   ${df_safmr['safmr_2br'].mean():,.0f}")
    print(f"    Median: ${df_safmr['safmr_2br'].median():,.0f}")
    print(f"    Std:    ${df_safmr['safmr_2br'].std():,.0f}")
    print(f"\n  Rent tiers:")
    for tier, count in df_safmr["rent_tier"].value_counts().sort_index().items():
        pct = count / len(df_safmr) * 100
        print(f"    {str(tier):12s}: {count:>6,} ZIPs ({pct:.1f}%)")

    # Save enhanced SAFMR with indicators
    safmr_enhanced_path = os.path.join(DATA_DIR, "processed", "hud_safmr_indicators.parquet")
    os.makedirs(os.path.dirname(safmr_enhanced_path), exist_ok=True)
    df_safmr.to_parquet(safmr_enhanced_path, index=False)
    print(f"\n  Saved: {safmr_enhanced_path}")
else:
    print("No SAFMR data available")

In [ ]:
# ---- SAFMR vs Census Rent Premium ----
# Load Census ACS state data to compute rent premium
print("Computing SAFMR vs Census rent premium...")
census_path = os.path.join(DATA_DIR, "census_state_demographics_2010_2024.csv")

if os.path.isfile(census_path) and len(df_safmr_state) > 0:
    df_census_st = pd.read_csv(census_path, dtype={"state": str})
    if "state_abbr" not in df_census_st.columns:
        df_census_st["state_abbr"] = df_census_st["state"].map(FIPS_TO_ABBR)
    latest_yr = int(df_census_st["year"].max())
    df_census_latest = df_census_st[df_census_st["year"] == latest_yr].copy()

    # Merge SAFMR with Census rent
    df_rent_compare = df_safmr_state.merge(
        df_census_latest[["state_abbr", "median_rent", "median_income", "median_home_value"]],
        on="state_abbr", how="inner"
    )

    if "median_rent" in df_rent_compare.columns:
        # SAFMR premium: how much HUD fair market rent differs from Census actual rent
        df_rent_compare["safmr_premium"] = df_rent_compare["safmr_2br_median"] - df_rent_compare["median_rent"]
        df_rent_compare["safmr_premium_pct"] = (
            df_rent_compare["safmr_premium"] / df_rent_compare["median_rent"].replace(0, np.nan) * 100
        )

        # Rent-to-income ratio (housing affordability stress indicator)
        df_rent_compare["rent_to_income_ratio"] = (
            df_rent_compare["safmr_2br_median"] * 12 / df_rent_compare["median_income"].replace(0, np.nan)
        )

        # Rent-to-home-value ratio (yield proxy)
        df_rent_compare["rent_to_value_ratio"] = (
            df_rent_compare["safmr_2br_median"] * 12 / df_rent_compare["median_home_value"].replace(0, np.nan)
        )

        print(f"\n--- SAFMR vs Census Rent Comparison ({latest_yr}, {len(df_rent_compare)} states) ---")
        print(f"  SAFMR premium: mean ${df_rent_compare['safmr_premium'].mean():,.0f} "
              f"({df_rent_compare['safmr_premium_pct'].mean():+.1f}%)")
        print(f"  Rent-to-income ratio: {df_rent_compare['rent_to_income_ratio'].mean():.2f} "
              f"(>0.30 = cost-burdened)")
        print(f"  Rent-to-value ratio: {df_rent_compare['rent_to_value_ratio'].mean():.3f} "
              f"(annual gross yield proxy)")

        # Top 5 most cost-burdened states
        top_burden = df_rent_compare.nlargest(5, "rent_to_income_ratio")
        print(f"\n  Most cost-burdened states (rent/income):")
        for _, r in top_burden.iterrows():
            print(f"    {r['state_abbr']}: {r['rent_to_income_ratio']:.2f} "
                  f"(SAFMR ${r['safmr_2br_median']:,.0f}, Income ${r['median_income']:,.0f})")

        df_rent_compare_path = os.path.join(DATA_DIR, "processed", "safmr_census_rent_comparison.parquet")
        df_rent_compare.to_parquet(df_rent_compare_path, index=False)
        print(f"\n  Saved: {df_rent_compare_path}")
    else:
        print("  Census median_rent column not found")
        df_rent_compare = pd.DataFrame()
else:
    print("  Census data or SAFMR state data not available")
    df_rent_compare = pd.DataFrame()

---
## 8. FHFA HPI -- State-Level Appreciation Rates for FMV

House price **appreciation rates** (1yr, 3yr, 5yr) are a strong temporal signal for FMV:
- Rising HPI = increasing land values = upward FMV pressure
- Falling HPI = depreciating market = downward FMV pressure

These rates capture **market momentum** at the state level from the quarterly time series.

In [ ]:
# ---- FHFA HPI: Compute State-Level Appreciation Rates ----
print("=" * 80)
print("FHFA HPI -- State-Level Appreciation Rates")
print("=" * 80)

if len(df_hpi_annual) > 0 and "year" in df_hpi_annual.columns:
    max_year = int(df_hpi_annual["year"].max())
    print(f"  Latest year: {max_year}")
    print(f"  States: {df_hpi_annual['state_abbr'].nunique()}")
    print(f"  Years available: {int(df_hpi_annual['year'].min())} - {max_year}")

    # Compute appreciation rates per state
    windows = {"1yr": 1, "3yr": 3, "5yr": 5, "10yr": 10}
    appreciation_records = []

    for state in df_hpi_annual["state_abbr"].unique():
        state_data = df_hpi_annual[df_hpi_annual["state_abbr"] == state].set_index("year")["hpi_annual_avg"]
        record = {"state_abbr": state, "hpi_latest": state_data.get(max_year)}

        for label, years_back in windows.items():
            compare_year = max_year - years_back
            if compare_year in state_data.index and max_year in state_data.index:
                latest_val = state_data[max_year]
                compare_val = state_data[compare_year]
                if compare_val > 0:
                    rate = (latest_val - compare_val) / compare_val
                    record[f"hpi_appreciation_{label}"] = round(rate, 4)
                    # Annualized rate
                    record[f"hpi_cagr_{label}"] = round((latest_val / compare_val) ** (1 / years_back) - 1, 4)

        appreciation_records.append(record)

    df_hpi_rates = pd.DataFrame(appreciation_records)

    print(f"\n  Appreciation rates computed for {len(df_hpi_rates)} states:")
    for col in [c for c in df_hpi_rates.columns if "appreciation" in c]:
        valid = df_hpi_rates[col].dropna()
        print(f"    {col:30s}: mean={valid.mean():+.3f}, median={valid.median():+.3f}, "
              f"range=[{valid.min():+.3f}, {valid.max():+.3f}]")

    # Top/bottom appreciating states
    if "hpi_appreciation_5yr" in df_hpi_rates.columns:
        print(f"\n  Top 5 appreciating states (5yr):")
        for _, r in df_hpi_rates.nlargest(5, "hpi_appreciation_5yr").iterrows():
            print(f"    {r['state_abbr']}: {r['hpi_appreciation_5yr']:+.1%}")
        print(f"  Bottom 5:")
        for _, r in df_hpi_rates.nsmallest(5, "hpi_appreciation_5yr").iterrows():
            print(f"    {r['state_abbr']}: {r['hpi_appreciation_5yr']:+.1%}")

    # Market momentum classification
    if "hpi_cagr_3yr" in df_hpi_rates.columns:
        df_hpi_rates["hpi_momentum"] = pd.cut(
            df_hpi_rates["hpi_cagr_3yr"],
            bins=[-1, -0.02, 0.02, 0.05, 0.10, 1],
            labels=["declining", "stagnant", "moderate_growth", "strong_growth", "boom"]
        )
        print(f"\n  Market momentum (3yr CAGR):")
        for cat, count in df_hpi_rates["hpi_momentum"].value_counts().sort_index().items():
            print(f"    {str(cat):20s}: {count} states")

    # Save
    hpi_rates_path = os.path.join(DATA_DIR, "processed", "fhfa_hpi_state_appreciation.parquet")
    os.makedirs(os.path.dirname(hpi_rates_path), exist_ok=True)
    df_hpi_rates.to_parquet(hpi_rates_path, index=False)
    print(f"\n  Saved: {hpi_rates_path}")
else:
    print("No FHFA HPI annual data available")
    df_hpi_rates = pd.DataFrame()

---
## 9. Wharton WRLURI -- Permit Difficulty Proxy for FMV

The WRLURI index directly measures **regulatory barriers** to construction, which maps to:
- `permit_difficulty` (FMV coefficient **0.08**): higher WRLURI = harder to permit = **higher FMV** (scarcity)
- `scarcity_index` (FMV coefficient **0.15**): restrictive zoning limits alternative tower sites

Sub-indices provide more granular signals:
- **LPPI** (Local Political Pressure Index): community opposition to development
- **LZAI** (Local Zoning Approval Index): number of approval bodies needed
- **DRI** (Density Restrictions Index): limits on density → limits on tower alternatives
- **SRI** (Supply Restrictions Index): caps on permits → scarcity

In [ ]:
# ---- WRLURI: Permit Difficulty Proxy for FMV ----
print("=" * 80)
print("Wharton WRLURI -- Permit Difficulty & Scarcity Proxy")
print("=" * 80)

if len(df_wrluri) > 0:
    # Normalize column names
    cols_upper = [c.upper() for c in df_wrluri.columns]
    df_w = df_wrluri.copy()
    df_w.columns = cols_upper

    # Find key sub-indices
    sub_idx_map = {}
    for idx_name in ["WRLURI", "LPPI", "SPII", "CII", "LZAI", "LPAI", "DRI", "SRI", "EI", "AHI"]:
        for col in df_w.columns:
            if idx_name == col or (idx_name in col and len(col) <= len(idx_name) + 4):
                sub_idx_map[idx_name] = col
                break

    print(f"  Sub-indices found: {list(sub_idx_map.keys())}")
    print(f"  Communities: {len(df_w):,}")

    # Convert to numeric
    for idx_name, col in sub_idx_map.items():
        df_w[col] = pd.to_numeric(df_w[col], errors="coerce")

    wrluri_col = sub_idx_map.get("WRLURI")
    if wrluri_col:
        # 1. Permit difficulty score: normalize WRLURI to 0-10 scale
        wrluri_vals = df_w[wrluri_col].dropna()
        wrluri_min, wrluri_max = wrluri_vals.quantile(0.01), wrluri_vals.quantile(0.99)
        df_w["permit_difficulty_score"] = (
            (df_w[wrluri_col] - wrluri_min) / (wrluri_max - wrluri_min) * 10
        ).clip(0, 10)

        # 2. Scarcity boost: more restrictive = harder to build alternatives = higher FMV
        df_w["wrluri_scarcity_boost"] = df_w["permit_difficulty_score"] * 0.15  # scaled to FMV weight

        # 3. Aggregate to state level with richer features
        state_col = None
        for candidate in ["STATENAME", "STATE", "STATE_ABBR"]:
            if candidate in df_w.columns:
                state_col = candidate
                break

        if state_col:
            agg_cols = {"permit_difficulty_score": ["mean", "median", "std", "count"]}
            for idx_name, col in sub_idx_map.items():
                if idx_name != "WRLURI":
                    agg_cols[col] = "mean"

            df_wrluri_fmv = df_w.groupby("STATE_ABBR" if "STATE_ABBR" in df_w.columns else state_col).agg(
                permit_difficulty_mean=("permit_difficulty_score", "mean"),
                permit_difficulty_median=("permit_difficulty_score", "median"),
                permit_difficulty_std=("permit_difficulty_score", "std"),
                wrluri_raw_mean=(wrluri_col, "mean"),
                community_count=(wrluri_col, "count"),
            ).reset_index()
            df_wrluri_fmv.columns = ["state_abbr"] + list(df_wrluri_fmv.columns[1:])

            # Map state names to abbreviations if needed
            if df_wrluri_fmv["state_abbr"].iloc[0] not in ABBR_TO_STATE_NAME:
                df_wrluri_fmv["state_abbr"] = df_wrluri_fmv["state_abbr"].map(STATE_NAME_TO_ABBR)

            # Scarcity boost at state level
            df_wrluri_fmv["wrluri_scarcity_boost"] = df_wrluri_fmv["permit_difficulty_mean"] * 0.15

            # Permit difficulty category
            df_wrluri_fmv["permit_category"] = pd.cut(
                df_wrluri_fmv["permit_difficulty_mean"],
                bins=[0, 3, 5, 7, 10],
                labels=["permissive", "moderate", "restrictive", "highly_restrictive"]
            )

            print(f"\n  State-Level Permit Difficulty ({len(df_wrluri_fmv)} states):")
            print(f"    Mean score: {df_wrluri_fmv['permit_difficulty_mean'].mean():.2f}/10")
            print(f"\n  Permit categories:")
            for cat, count in df_wrluri_fmv["permit_category"].value_counts().sort_index().items():
                states = df_wrluri_fmv[df_wrluri_fmv["permit_category"] == cat]["state_abbr"].tolist()
                print(f"    {str(cat):22s}: {count} states ({', '.join(states[:5])}{'...' if len(states) > 5 else ''})")

            print(f"\n  Most restrictive states:")
            for _, r in df_wrluri_fmv.nlargest(5, "permit_difficulty_mean").iterrows():
                print(f"    {r['state_abbr']}: {r['permit_difficulty_mean']:.2f}/10 "
                      f"({r['community_count']:.0f} communities)")

            print(f"\n  Most permissive states:")
            for _, r in df_wrluri_fmv.nsmallest(5, "permit_difficulty_mean").iterrows():
                print(f"    {r['state_abbr']}: {r['permit_difficulty_mean']:.2f}/10")

            # Save
            wrluri_fmv_path = os.path.join(DATA_DIR, "processed", "wrluri_permit_difficulty.parquet")
            df_wrluri_fmv.to_parquet(wrluri_fmv_path, index=False)
            print(f"\n  Saved: {wrluri_fmv_path}")
        else:
            print("  State column not found in WRLURI data")
            df_wrluri_fmv = pd.DataFrame()
    else:
        print("  WRLURI column not found")
        df_wrluri_fmv = pd.DataFrame()
else:
    print("No WRLURI data available")
    df_wrluri_fmv = pd.DataFrame()

---
## 10. Composite Housing Market Features for FMV

Combine all three housing sources into **composite indicator variables** ready for the unified dataset.

In [ ]:
# ---- Composite Housing Market Features ----
print("=" * 80)
print("Composite Housing Market Features for FMV Model")
print("=" * 80)

# Build state-level composite from all three sources
df_housing_fmv = pd.DataFrame({"state_abbr": list(ABBR_TO_FIPS.keys())})

# 1. HUD SAFMR features
if len(df_safmr_state) > 0:
    safmr_cols = ["state_abbr", "safmr_2br_median", "rent_spread_4br_0br"]
    avail = [c for c in safmr_cols if c in df_safmr_state.columns]
    df_housing_fmv = df_housing_fmv.merge(df_safmr_state[avail], on="state_abbr", how="left")
    print(f"  Added SAFMR: {len(avail) - 1} features")

# 2. SAFMR vs Census rent comparison
if len(df_rent_compare) > 0:
    rent_cols = ["state_abbr", "safmr_premium", "safmr_premium_pct",
                 "rent_to_income_ratio", "rent_to_value_ratio"]
    avail = [c for c in rent_cols if c in df_rent_compare.columns]
    df_housing_fmv = df_housing_fmv.merge(df_rent_compare[avail], on="state_abbr", how="left")
    print(f"  Added rent comparison: {len(avail) - 1} features")

# 3. FHFA HPI appreciation rates
if len(df_hpi_rates) > 0:
    hpi_cols = ["state_abbr", "hpi_latest", "hpi_appreciation_1yr", "hpi_appreciation_3yr",
                "hpi_appreciation_5yr", "hpi_cagr_3yr", "hpi_momentum"]
    avail = [c for c in hpi_cols if c in df_hpi_rates.columns]
    df_housing_fmv = df_housing_fmv.merge(df_hpi_rates[avail], on="state_abbr", how="left")
    print(f"  Added HPI appreciation: {len(avail) - 1} features")

# 4. WRLURI permit difficulty
if len(df_wrluri_fmv) > 0:
    wrluri_cols = ["state_abbr", "permit_difficulty_mean", "permit_difficulty_median",
                   "wrluri_scarcity_boost", "permit_category"]
    avail = [c for c in wrluri_cols if c in df_wrluri_fmv.columns]
    df_housing_fmv = df_housing_fmv.merge(df_wrluri_fmv[avail], on="state_abbr", how="left")
    print(f"  Added WRLURI: {len(avail) - 1} features")

# --- Derived composite features ---

# Housing market pressure index (0-10): combines rent level, appreciation, and affordability stress
rent_pctile = df_housing_fmv["safmr_2br_median"].rank(pct=True).fillna(0.5) * 3
hpi_pctile = df_housing_fmv.get("hpi_appreciation_3yr", pd.Series(dtype=float)).rank(pct=True).fillna(0.5) * 3
afford_stress = df_housing_fmv.get("rent_to_income_ratio", pd.Series(dtype=float)).rank(pct=True).fillna(0.5) * 4
df_housing_fmv["housing_market_pressure"] = (rent_pctile + hpi_pctile + afford_stress).clip(0, 10)

# Land value proxy (log scale) -- combines rent level and HPI
if "safmr_2br_median" in df_housing_fmv.columns:
    df_housing_fmv["log_land_value_proxy"] = np.log1p(
        df_housing_fmv["safmr_2br_median"].fillna(0) * 12
    )

# FMV upward pressure score: higher = more pressure for higher FMV
# permit_difficulty pushes FMV UP (scarcity), appreciation pushes UP (rising market)
permit_score = df_housing_fmv.get("permit_difficulty_mean", pd.Series(dtype=float)).fillna(5)
appreciation_score = df_housing_fmv.get("hpi_appreciation_3yr", pd.Series(dtype=float)).rank(pct=True).fillna(0.5) * 10
df_housing_fmv["fmv_upward_pressure"] = ((permit_score + appreciation_score) / 2).clip(0, 10)

# Drop rows with no data at all
feature_cols = [c for c in df_housing_fmv.columns if c != "state_abbr"]
df_housing_fmv = df_housing_fmv.dropna(subset=feature_cols, how="all")

print(f"\n--- Composite Housing Features: {len(df_housing_fmv)} states x {len(df_housing_fmv.columns)} columns ---")
print(f"\n  All features:")
for col in feature_cols:
    if df_housing_fmv[col].dtype in ["float64", "int64", "Float64"]:
        valid = df_housing_fmv[col].dropna()
        print(f"    {col:35s}: mean={valid.mean():.3f}, median={valid.median():.3f}, coverage={len(valid)}/{len(df_housing_fmv)}")
    else:
        print(f"    {col:35s}: {df_housing_fmv[col].nunique()} unique values")

# Save
composite_path = os.path.join(DATA_DIR, "processed", "housing_market_composites.parquet")
df_housing_fmv.to_parquet(composite_path, index=False)
print(f"\n  Saved: {composite_path}")
df_housing_fmv.head(10)

In [ ]:
# ---- Map 6: Housing Market Pressure Index by State ----
if "housing_market_pressure" in df_housing_fmv.columns:
    fig_pressure = px.choropleth(
        df_housing_fmv.dropna(subset=["housing_market_pressure"]),
        locations="state_abbr",
        locationmode="USA-states",
        color="housing_market_pressure",
        scope="usa",
        color_continuous_scale="RdYlGn_r",
        range_color=[0, 10],
        hover_name="state_abbr",
        hover_data={
            "housing_market_pressure": ":.1f",
            "safmr_2br_median": ":$,.0f",
            "hpi_appreciation_3yr": ":.1%",
            "permit_difficulty_mean": ":.1f",
            "state_abbr": False,
        },
        labels={"housing_market_pressure": "Pressure (0-10)"},
        title="Housing Market Pressure Index by State (rent level + HPI appreciation + affordability stress)",
    )
    fig_pressure.update_layout(
        geo=dict(bgcolor="rgba(0,0,0,0)"),
        paper_bgcolor="white",
        margin=dict(l=0, r=0, t=50, b=0),
        height=500,
    )
    fig_pressure.show()

# ---- Map 7: FMV Upward Pressure by State ----
if "fmv_upward_pressure" in df_housing_fmv.columns:
    fig_fmv_press = px.choropleth(
        df_housing_fmv.dropna(subset=["fmv_upward_pressure"]),
        locations="state_abbr",
        locationmode="USA-states",
        color="fmv_upward_pressure",
        scope="usa",
        color_continuous_scale="Reds",
        range_color=[0, 10],
        hover_name="state_abbr",
        hover_data={
            "fmv_upward_pressure": ":.1f",
            "permit_difficulty_mean": ":.1f",
            "hpi_appreciation_3yr": ":.1%",
            "state_abbr": False,
        },
        labels={"fmv_upward_pressure": "FMV Pressure (0-10)"},
        title="FMV Upward Pressure by State (permit difficulty + appreciation momentum)",
    )
    fig_fmv_press.update_layout(
        geo=dict(bgcolor="rgba(0,0,0,0)"),
        paper_bgcolor="white",
        margin=dict(l=0, r=0, t=50, b=0),
        height=500,
    )
    fig_fmv_press.show()

---
## 11. Feature-to-FMV Mapping -- Housing & Regulatory Sources

### Raw Features → FMV Model Coefficients

| Raw Feature | Source | Granularity | FMV Coefficient | Mechanism |
|-------------|--------|-------------|-----------------|-----------|
| `safmr_2br` | HUD SAFMR | **ZIP** | `log_land_value` (0.15) | Government-set rent floor reflects local market value |
| `log_safmr_2br` | HUD SAFMR | ZIP | `log_land_value` (0.15) | Log-scale for FMV model |
| `safmr_premium` | HUD vs Census | State | demand signal | SAFMR > Census rent = strong demand market |
| `rent_to_income_ratio` | HUD + Census | State | `log_land_value` (0.15) | Affordability stress = high-demand area = higher land value |
| `rent_to_value_ratio` | HUD + Census | State | yield proxy | Gross rental yield; higher = more cash-flow-driven market |
| `hpi_appreciation_1yr/3yr/5yr` | FHFA HPI | State | `log_median_home_value` (0.35) | Rising prices = rising FMV |
| `hpi_cagr_3yr` | FHFA HPI | State | temporal momentum | Annualized growth rate captures trend strength |
| `hpi_momentum` | FHFA HPI | State | market regime | Categorical: declining/stagnant/moderate/strong/boom |
| `permit_difficulty_mean` | WRLURI | State | `permit_difficulty` (0.08) | **Direct proxy**: higher = harder to permit = higher FMV |
| `wrluri_scarcity_boost` | WRLURI | State | `scarcity_index` (0.15) | Restrictive regulation limits alternatives = scarcity premium |

### Composite Features → Direct Model Inputs

| Composite | Scale | Components | Use |
|-----------|-------|------------|-----|
| `housing_market_pressure` | 0-10 | Rent level + HPI appreciation + affordability | Overall market heat → predicts FMV level |
| `log_land_value_proxy` | log scale | Annualized SAFMR (2BR x 12) | Direct substitute for `log_land_value` coefficient |
| `fmv_upward_pressure` | 0-10 | Permit difficulty + appreciation | Predicts FMV premium over baseline |
| `rent_tier` | categorical | Budget/moderate/above_avg/premium/luxury | Market segment for clustering |
| `permit_category` | categorical | Permissive/moderate/restrictive/highly_restrictive | Regulatory regime |

### Files Produced

| File | Grain | Join Key | Contents |
|------|-------|----------|----------|
| `data/processed/hud_safmr_indicators.parquet` | per ZIP | `zcta` → `zip_code` | 33,800 ZIPs with rent tier, percentile, log rent |
| `data/processed/safmr_census_rent_comparison.parquet` | per state | `state_abbr` | Rent premium, affordability ratios |
| `data/processed/fhfa_hpi_state_appreciation.parquet` | per state | `state_abbr` | 1yr/3yr/5yr appreciation, CAGR, momentum |
| `data/processed/wrluri_permit_difficulty.parquet` | per state | `state_abbr` | Permit difficulty score, scarcity boost, category |
| `data/processed/housing_market_composites.parquet` | per state | `state_abbr` | All composites: market pressure, FMV pressure, land value proxy |